# CycleGAN


Ref: CHU, Casey; ZHMOGINOV, Andrey; SANDLER, Mark. Cyclegan, a master of steganography. arXiv preprint arXiv:1712.02950, 2017.

In [1]:
# load libraries
import datetime
import matplotlib.pyplot as plt
import numpy as np
import os

In [2]:
# from keras.datasets import mnist
from keras_contrib.layers.normalization.instancenormalization import InstanceNormalization
from keras.layers import Input, Dropout, Concatenate
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Model
from keras.optimizers import Adam

Using TensorFlow backend.


## Data loader

In [3]:
from data_loader2 import DataLoader

## Helper function for the generator

In [4]:
def conv2d(layer_input, filters, f_size=4, strides=2, alpha=0.2):
    """Layers used during downsampling"""
    d = Conv2D(filters, kernel_size=f_size, strides=strides, padding='same')(layer_input)
    d = LeakyReLU(alpha=alpha)(d)
    d = InstanceNormalization()(d)
    return d

def deconv2d(layer_input, skip_input, filters, f_size=4, dropout_rate=0, up_size=2, strides=1):
    """ Layers used during upsampling
        :param skip_input: layer to make skip connection to
    """
    u = UpSampling2D(size=up_size)(layer_input)
    u = Conv2D(filters, kernel_size=f_size, strides=strides, padding='same', activation='relu')(u)
    if dropout_rate:
        u = Dropout(dropout_rate)(u)
    u = InstanceNormalization()(u)
    u = Concatenate()([u, skip_input])
    return u

## Generator

In [5]:
def build_generator(img_shape, gf, channels):
    """ U-Net Generator
        :param gf: generator filter size in the first layer
        
        
        ref.: RONNEBERGER, Olaf; FISCHER, Philipp; BROX, Thomas. U-net: Convolutional networks for biomedical image segmentation. 
              In: International Conference on Medical image computing and computer-assisted intervention. 
              Springer, Cham, 2015. p. 234-241.
    """

    # Image input
    d0 = Input(shape=img_shape)

    # Downsampling
    d1 = conv2d(d0, gf)
    d2 = conv2d(d1, gf*2)
    d3 = conv2d(d2, gf*4)
    d4 = conv2d(d3, gf*8)

    # Upsampling
    u1 = deconv2d(d4, d3, gf*4)
    u2 = deconv2d(u1, d2, gf*2)
    u3 = deconv2d(u2, d1, gf)

    u4 = UpSampling2D(size=2)(u3)
    output_img = Conv2D(channels, kernel_size=4, strides=1, padding='same', activation='tanh')(u4)

    return Model(d0, output_img)

## Helper function for Discriminator

In [6]:
def d_layer(layer_input, filters, f_size=4, normalization=True, strides=2):
    """Discriminator layer"""
    d = Conv2D(filters, kernel_size=f_size, strides=strides, padding='same')(layer_input)
    d = LeakyReLU(alpha=0.2)(d)
    if normalization:
        d = InstanceNormalization()(d)
    return d

# Discriminator

In [7]:
def build_discriminator(img_shape, df):

    img = Input(shape=img_shape)

    d1 = d_layer(img, df, normalization=False)
    d2 = d_layer(d1, df*2)
    d3 = d_layer(d2, df*4)
    d4 = d_layer(d3, df*8)

    validity = Conv2D(1, kernel_size=4, strides=1, padding='same')(d4)

    return Model(img, validity)

In [ ]:
def sample_images(epoch, batch_i,
                  g_AB, g_BA, # generators
                  dataset_name, data_loader):
    os.makedirs('images/%s' % dataset_name, exist_ok=True)
    r, c = 2, 3 # rows, columns

    imgs_A = data_loader.load_data(domain="A", batch_size=1, is_testing=True)
    imgs_B = data_loader.load_data(domain="B", batch_size=1, is_testing=True)

    # Translate images to the other domain
    fake_B = g_AB.predict(imgs_A)
    fake_A = g_BA.predict(imgs_B)
    # Translate back to original domain
    reconstr_A = g_BA.predict(fake_B)
    reconstr_B = g_AB.predict(fake_A)

    gen_imgs = np.concatenate([imgs_A, fake_B, reconstr_A, imgs_B, fake_A, reconstr_B])

    # Rescale images 0 - 1
    gen_imgs = 0.5 * gen_imgs + 0.5

    titles = ['Original', 'Translated', 'Reconstructed']
    fig, axs = plt.subplots(r, c)
    cnt = 0
    for i in range(r):  # rows
        for j in range(c):  # columns
            axs[i,j].imshow(gen_imgs[cnt])
            axs[i, j].set_title(titles[j])
            axs[i,j].axis('off')
            cnt += 1
    fig.savefig("images/%s unet/%d_%d.png" % (dataset_name, epoch, batch_i))
    plt.close()

## Perform training of the GAN

In [9]:
def train(epochs, 
          g_AB, g_BA, # generators
          d_A, d_B, # discriminators
          combined, # full GAN
          dataset_name, img_rows, img_cols,
          batch_size=1, sample_interval=50,
          ):
    
    # Calculate output shape of D (PatchGAN)
    patch = int(img_rows / 2**4)
    disc_patch = (patch, patch, 1)

    # class that loads the data in batches
    data_loader = DataLoader(dataset_name=dataset_name,
                             img_res=(img_rows, img_cols))

    start_time = datetime.datetime.now()

    # Adversarial loss ground truths
    valid = np.ones((batch_size,) + disc_patch)
    fake = np.zeros((batch_size,) + disc_patch)

    for epoch in range(epochs):
        for batch_i, (imgs_A, imgs_B) in enumerate(data_loader.load_batch(batch_size)):

            # ----------------------
            #  Train Discriminators
            # ----------------------

            # Translate images to opposite domain
            fake_B = g_AB.predict(imgs_A)
            fake_A = g_BA.predict(imgs_B)

            # Train the discriminators (original images = real / translated = Fake)
            dA_loss_real = d_A.train_on_batch(imgs_A, valid)
            dA_loss_fake = d_A.train_on_batch(fake_A, fake)
            dA_loss = 0.5 * np.add(dA_loss_real, dA_loss_fake)

            dB_loss_real = d_B.train_on_batch(imgs_B, valid)
            dB_loss_fake = d_B.train_on_batch(fake_B, fake)
            dB_loss = 0.5 * np.add(dB_loss_real, dB_loss_fake)

            # Total disciminator loss
            d_loss = 0.5 * np.add(dA_loss, dB_loss)


            # ------------------
            #  Train Generators
            # ------------------

            # Train the generators
            g_loss = combined.train_on_batch([imgs_A, imgs_B],
                                             [valid, valid,
                                              imgs_A, imgs_B,
                                              imgs_A, imgs_B])

            elapsed_time = datetime.datetime.now() - start_time

            # Plot the progress
            print ("[Epoch %d/%d] [Batch %d/%d] [D loss: %f, acc: %3d%%] [G loss: %05f, adv: %05f, recon: %05f, id: %05f] time: %s " \
                                                                    % ( epoch, epochs,
                                                                        batch_i, data_loader.n_batches,
                                                                        d_loss[0], 100*d_loss[1],
                                                                        g_loss[0],
                                                                        np.mean(g_loss[1:3]),
                                                                        np.mean(g_loss[3:5]),
                                                                        np.mean(g_loss[5:6]),
                                                                        elapsed_time))

            # If at save interval => save generated image samples
            if batch_i % sample_interval == 0:
                sample_images(epoch, batch_i, 
                              g_AB, g_BA, # generators
                              dataset_name, data_loader)

## main()

In [10]:
img_rows = 128
img_cols = 128
channels = 3
img_shape = (img_rows, img_cols, channels)

# Configure data loader
dataset_name = 'apple2orange'

In [11]:
# Loss weights
lambda_cycle = 10.0               # Cycle-consistency loss
lambda_id = 0.1 * lambda_cycle    # Identity loss

In [12]:
# optimizer
optimizer = Adam(0.0002, 0.5)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [13]:
# Number of filters in the first layer of G and D
gf = 32
df = 64

In [14]:
# Build and compile the discriminators
d_A = build_discriminator(img_shape, df)
d_B = build_discriminator(img_shape, df)
d_A.compile(loss='mse',
    optimizer=optimizer,
    metrics=['accuracy'])
d_B.compile(loss='mse',
    optimizer=optimizer,
    metrics=['accuracy'])

In [15]:
#-------------------------
# Construct Computational
#   Graph of Generators
#-------------------------

# Build the generators
g_AB = build_generator(img_shape, gf, channels)
g_BA = build_generator(img_shape, gf, channels)

In [16]:
# Input images from both domains
img_A = Input(shape=img_shape)
img_B = Input(shape=img_shape)

In [17]:
# Translate images to the other domain
fake_B = g_AB(img_A)
fake_A = g_BA(img_B)

In [18]:
# Translate images back to original domain
reconstr_A = g_BA(fake_B)
reconstr_B = g_AB(fake_A)

In [19]:
# Identity mapping of images
img_A_id = g_BA(img_A)
img_B_id = g_AB(img_B)

In [20]:
# For the combined model we will only train the generators
d_A.trainable = False
d_B.trainable = False

# Discriminators determines validity of translated images
valid_A = d_A(fake_A)
valid_B = d_B(fake_B)

In [21]:
# Combined model trains generators to fool discriminators
combined = Model(inputs=[img_A, img_B],
                      outputs=[ valid_A, valid_B,
                                reconstr_A, reconstr_B,
                                img_A_id, img_B_id ])
combined.compile(loss=['mse', 'mse',
                            'mae', 'mae',
                            'mae', 'mae'],
                    loss_weights=[  1, 1,
                                    lambda_cycle, lambda_cycle,
                                    lambda_id, lambda_id ],
                    optimizer=optimizer)

In [ ]:
epochs = 200
train(epochs=epochs, batch_size=1, sample_interval=200,
      g_AB=g_AB, g_BA=g_BA, d_A=d_A, d_B=d_B, combined=combined,
      dataset_name=dataset_name, img_rows=img_rows, img_cols=img_cols
      )

c:\users\henri\appdata\local\programs\python\python37\lib\site-packages\keras\engine\training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'
c:\users\henri\appdata\local\programs\python\python37\lib\site-packages\keras\engine\training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'
c:\users\henri\appdata\local\programs\python\python37\lib\site-packages\keras\engine\training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


[Epoch 0/200] [Batch 0/995] [D loss: 8.486359, acc:  13%] [G loss: 174.135864, adv: 77.569778, recon: 0.861589, id: 0.953670] time: 0:00:19.937361 


c:\users\henri\appdata\local\programs\python\python37\lib\site-packages\keras\engine\training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


[Epoch 0/200] [Batch 1/995] [D loss: 42.934010, acc:   2%] [G loss: 95.517090, adv: 41.090157, recon: 0.593035, id: 0.882590] time: 0:00:21.321363 
[Epoch 0/200] [Batch 2/995] [D loss: 23.606171, acc:   6%] [G loss: 130.560150, adv: 58.708984, recon: 0.592322, id: 0.896176] time: 0:00:22.602363 
[Epoch 0/200] [Batch 3/995] [D loss: 28.504810, acc:  16%] [G loss: 21.649271, adv: 4.010476, recon: 0.610589, id: 0.835317] time: 0:00:23.880364 
[Epoch 0/200] [Batch 4/995] [D loss: 12.093575, acc:  15%] [G loss: 17.656387, adv: 2.937593, recon: 0.503098, id: 0.805716] time: 0:00:25.149364 
[Epoch 0/200] [Batch 5/995] [D loss: 4.111576, acc:  29%] [G loss: 14.743592, adv: 1.663451, recon: 0.486281, id: 0.893659] time: 0:00:26.422362 
[Epoch 0/200] [Batch 6/995] [D loss: 2.130726, acc:  19%] [G loss: 14.517710, adv: 0.349985, recon: 0.613137, id: 0.849806] time: 0:00:27.697363 
[Epoch 0/200] [Batch 7/995] [D loss: 0.699325, acc:  40%] [G loss: 12.890071, adv: 0.771648, recon: 0.472224, id: 1.1

[Epoch 0/200] [Batch 57/995] [D loss: 0.246898, acc:  64%] [G loss: 12.233382, adv: 0.811741, recon: 0.455903, id: 0.769958] time: 0:01:38.503710 
[Epoch 0/200] [Batch 58/995] [D loss: 0.286238, acc:  64%] [G loss: 10.196877, adv: 0.837490, recon: 0.329670, id: 1.147726] time: 0:01:39.816711 
[Epoch 0/200] [Batch 59/995] [D loss: 0.291114, acc:  47%] [G loss: 8.356262, adv: 0.804298, recon: 0.242911, id: 1.198769] time: 0:01:41.100709 
[Epoch 0/200] [Batch 60/995] [D loss: 0.471684, acc:  48%] [G loss: 12.068276, adv: 1.028798, recon: 0.420800, id: 0.905163] time: 0:01:42.401711 
[Epoch 0/200] [Batch 61/995] [D loss: 0.293976, acc:  58%] [G loss: 9.030931, adv: 0.786770, recon: 0.249992, id: 1.300161] time: 0:01:43.742711 
[Epoch 0/200] [Batch 62/995] [D loss: 0.380831, acc:  53%] [G loss: 9.390021, adv: 0.711950, recon: 0.309845, id: 1.044399] time: 0:01:45.051711 
[Epoch 0/200] [Batch 63/995] [D loss: 0.363821, acc:  50%] [G loss: 8.724847, adv: 0.999818, recon: 0.257889, id: 1.02141

...

[Epoch 11/200] [Batch 392/995] [D loss: 0.104341, acc:  88%] [G loss: 5.501966, adv: 0.724151, recon: 0.153238, id: 0.169151] time: 4:05:47.838272 
[Epoch 11/200] [Batch 393/995] [D loss: 0.143864, acc:  87%] [G loss: 3.865201, adv: 0.580270, recon: 0.096310, id: 0.321065] time: 4:05:49.034272 
[Epoch 11/200] [Batch 394/995] [D loss: 0.291716, acc:  57%] [G loss: 4.713677, adv: 0.589316, recon: 0.120271, id: 0.246333] time: 4:05:50.211270 
[Epoch 11/200] [Batch 395/995] [D loss: 0.151379, acc:  73%] [G loss: 4.682421, adv: 0.784845, recon: 0.105689, id: 0.128256] time: 4:05:51.380270 
[Epoch 11/200] [Batch 396/995] [D loss: 0.387713, acc:  27%] [G loss: 3.132431, adv: 0.393819, recon: 0.076186, id: 0.105375] time: 4:05:52.543271 
[Epoch 11/200] [Batch 397/995] [D loss: 0.257855, acc:  61%] [G loss: 2.982232, adv: 0.405310, recon: 0.080208, id: 0.143415] time: 4:05:53.700271 
[Epoch 11/200] [Batch 398/995] [D loss: 0.074398, acc:  91%] [G loss: 4.288340, adv: 0.862442, recon: 0.089093, 

[Epoch 11/200] [Batch 448/995] [D loss: 0.120290, acc:  75%] [G loss: 3.574860, adv: 0.874727, recon: 0.060096, id: 0.128376] time: 4:06:55.937411 
[Epoch 11/200] [Batch 449/995] [D loss: 0.346705, acc:  41%] [G loss: 4.083247, adv: 0.343761, recon: 0.096741, id: 0.193513] time: 4:06:57.090413 
[Epoch 11/200] [Batch 450/995] [D loss: 0.111662, acc:  95%] [G loss: 5.610809, adv: 0.610314, recon: 0.156817, id: 0.206415] time: 4:06:58.266412 
[Epoch 11/200] [Batch 451/995] [D loss: 0.038125, acc:  99%] [G loss: 6.057076, adv: 0.828511, recon: 0.177144, id: 0.256296] time: 4:06:59.339411 
[Epoch 11/200] [Batch 452/995] [D loss: 0.127398, acc:  78%] [G loss: 5.017636, adv: 0.687580, recon: 0.150234, id: 0.081584] time: 4:07:00.412411 
[Epoch 11/200] [Batch 453/995] [D loss: 0.095543, acc:  96%] [G loss: 5.055905, adv: 0.604725, recon: 0.110933, id: 1.238484] time: 4:07:01.477410 
[Epoch 11/200] [Batch 454/995] [D loss: 0.140666, acc:  76%] [G loss: 5.673282, adv: 0.673059, recon: 0.164940, 

[Epoch 11/200] [Batch 504/995] [D loss: 0.320211, acc:  28%] [G loss: 2.732666, adv: 0.318767, recon: 0.078964, id: 0.127707] time: 4:08:02.185599 
[Epoch 11/200] [Batch 505/995] [D loss: 0.119659, acc:  89%] [G loss: 3.893022, adv: 0.560238, recon: 0.104030, id: 0.187148] time: 4:08:03.381598 
[Epoch 11/200] [Batch 506/995] [D loss: 0.243335, acc:  53%] [G loss: 4.463994, adv: 0.485438, recon: 0.087915, id: 0.913473] time: 4:08:04.640601 
[Epoch 11/200] [Batch 507/995] [D loss: 0.214903, acc:  66%] [G loss: 3.661318, adv: 0.578831, recon: 0.079885, id: 0.156077] time: 4:08:05.996599 
[Epoch 11/200] [Batch 508/995] [D loss: 0.155245, acc:  65%] [G loss: 3.407832, adv: 0.648136, recon: 0.061029, id: 0.075614] time: 4:08:07.361599 
[Epoch 11/200] [Batch 509/995] [D loss: 0.187968, acc:  76%] [G loss: 4.156217, adv: 0.455017, recon: 0.082775, id: 0.161426] time: 4:08:08.832601 
[Epoch 11/200] [Batch 510/995] [D loss: 0.248991, acc:  68%] [G loss: 3.961161, adv: 0.415967, recon: 0.089449, 

[Epoch 11/200] [Batch 560/995] [D loss: 0.209565, acc:  58%] [G loss: 4.906994, adv: 0.618164, recon: 0.135544, id: 0.350108] time: 4:09:22.091930 
[Epoch 11/200] [Batch 561/995] [D loss: 0.333940, acc:  51%] [G loss: 4.211163, adv: 0.453390, recon: 0.080977, id: 0.880628] time: 4:09:23.679462 
[Epoch 11/200] [Batch 562/995] [D loss: 0.099790, acc:  91%] [G loss: 3.417014, adv: 0.749013, recon: 0.067722, id: 0.408160] time: 4:09:25.125463 
[Epoch 11/200] [Batch 563/995] [D loss: 0.067083, acc:  98%] [G loss: 3.823097, adv: 0.732913, recon: 0.077493, id: 0.151742] time: 4:09:26.498980 
[Epoch 11/200] [Batch 564/995] [D loss: 0.048863, acc:  99%] [G loss: 4.727070, adv: 0.864560, recon: 0.092909, id: 0.384187] time: 4:09:27.850496 
[Epoch 11/200] [Batch 565/995] [D loss: 0.166003, acc:  73%] [G loss: 3.544420, adv: 0.563139, recon: 0.087198, id: 0.169864] time: 4:09:29.202192 
[Epoch 11/200] [Batch 566/995] [D loss: 0.244433, acc:  55%] [G loss: 4.601442, adv: 0.433087, recon: 0.148844, 

[Epoch 11/200] [Batch 616/995] [D loss: 0.075195, acc:  94%] [G loss: 3.351037, adv: 0.846058, recon: 0.060087, id: 0.060285] time: 4:10:39.876837 
[Epoch 11/200] [Batch 617/995] [D loss: 0.525831, acc:  42%] [G loss: 4.507007, adv: 0.413889, recon: 0.128818, id: 0.217011] time: 4:10:41.341829 
[Epoch 11/200] [Batch 618/995] [D loss: 0.159911, acc:  82%] [G loss: 4.991635, adv: 0.744886, recon: 0.083746, id: 0.560423] time: 4:10:42.570829 
[Epoch 11/200] [Batch 619/995] [D loss: 0.424289, acc:  23%] [G loss: 4.075656, adv: 0.351326, recon: 0.125514, id: 0.256948] time: 4:10:43.774827 
[Epoch 11/200] [Batch 620/995] [D loss: 0.108879, acc:  90%] [G loss: 5.296867, adv: 0.694576, recon: 0.101533, id: 0.831199] time: 4:10:44.956829 
[Epoch 11/200] [Batch 621/995] [D loss: 0.348403, acc:  30%] [G loss: 3.630101, adv: 0.361129, recon: 0.109939, id: 0.169490] time: 4:10:46.465829 
[Epoch 11/200] [Batch 622/995] [D loss: 0.214948, acc:  54%] [G loss: 3.607587, adv: 0.579476, recon: 0.065104, 